# AILib Tutorial 7: Tools and Decorators

Tools extend the capabilities of AI agents by giving them access to functions they can call. In this tutorial, you'll learn:

- Creating tools with the @tool decorator
- Manual tool registration
- Tool parameters and validation
- Building custom tool registries
- Advanced tool patterns
- Best practices for tool design

## Setup

Let's import what we need:

In [ ]:
from ailib import OpenAIClient
from ailib.agents import Tool, ToolRegistry, tool, get_global_registry
from pydantic import BaseModel, Field
from typing import Optional
from datetime import datetime
from dotenv import load_dotenv
import json
import math

# Load environment variables
load_dotenv()

# Create a client
client = OpenAIClient()
print("Ready to create tools!")

## Basic Tool Creation with @tool Decorator

The simplest way to create tools is with the @tool decorator:

In [ ]:
# Create a custom registry to avoid conflicts in Jupyter
my_registry = ToolRegistry()

# Simple tool with decorator
@tool(registry=my_registry)
def get_current_time() -> str:
    """Get the current date and time."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# Tool with parameters
@tool(registry=my_registry)
def calculate(expression: str) -> float:
    """Evaluate a mathematical expression.
    
    Args:
        expression: Mathematical expression to evaluate
        
    Returns:
        Result of the calculation
    """
    # Safe evaluation
    allowed = {"__builtins__": {}}
    allowed.update({name: getattr(math, name) for name in dir(math) if not name.startswith("_")})
    return float(eval(expression, allowed))

# Tool with custom name and description
@tool(name="weather_info", description="Get weather for a city", registry=my_registry)
def get_weather(city: str, units: str = "celsius") -> str:
    """Get weather information for a city."""
    # Mock implementation
    temps = {"celsius": "22°C", "fahrenheit": "72°F"}
    return f"Weather in {city}: Sunny, {temps.get(units, '22°C')}"

# List registered tools
print("Registered tools:")
for tool_name in my_registry.list_tools():
    tool_obj = my_registry.get(tool_name)
    print(f"- {tool_name}: {tool_obj.description}")

# Test the tools
print("\nTesting tools:")
print(f"Current time: {get_current_time()}")
print(f"Calculation: 5 * 7 + 3 = {calculate('5 * 7 + 3')}")
print(f"Weather: {get_weather('London', 'celsius')}")

## Manual Tool Registration

For more control, create tools manually:

In [ ]:
# Create a new registry for manual tools
manual_registry = ToolRegistry()

# Define tool functions
def search_web(query: str, max_results: int = 5) -> list:
    """Search the web for information."""
    # Mock implementation
    results = []
    for i in range(min(max_results, 3)):
        results.append({
            "title": f"Result {i+1} for '{query}'",
            "url": f"https://example.com/{i+1}",
            "snippet": f"This is a snippet about {query}..."
        })
    return results

def send_email(to: str, subject: str, body: str) -> dict:
    """Send an email."""
    # Mock implementation
    return {
        "status": "sent",
        "to": to,
        "subject": subject,
        "timestamp": datetime.now().isoformat()
    }

# Create Tool objects manually
search_tool = Tool(
    name="search_web",
    description="Search the web for information",
    func=search_web,
    metadata={"category": "information", "requires_internet": True}
)

email_tool = Tool(
    name="send_email",
    description="Send an email message",
    func=send_email,
    metadata={"category": "communication", "requires_auth": True}
)

# Register tools
manual_registry.register(search_tool)
manual_registry.register(email_tool)

# Execute tools through registry
print("Executing search:")
search_results = manual_registry.execute_tool("search_web", query="AI agents", max_results=2)
print(json.dumps(search_results, indent=2))

print("\nExecuting email:")
email_result = manual_registry.execute_tool(
    "send_email",
    to="user@example.com",
    subject="Test Email",
    body="This is a test email from AILib."
)
print(json.dumps(email_result, indent=2))

## Tools with Pydantic Validation

Use Pydantic models for parameter validation:

In [ ]:
# Define Pydantic models for parameters
class FileOperationParams(BaseModel):
    path: str = Field(description="File path")
    content: Optional[str] = Field(None, description="File content for write operations")
    mode: str = Field("r", description="File mode (r, w, a)")

class DataQueryParams(BaseModel):
    table: str = Field(description="Table name")
    filters: dict = Field(default_factory=dict, description="Query filters")
    limit: int = Field(10, ge=1, le=100, description="Result limit")
    order_by: Optional[str] = Field(None, description="Order by field")

# Create a registry for validated tools
validated_registry = ToolRegistry()

# Tool with Pydantic validation
@tool(registry=validated_registry)
def file_operation(params: FileOperationParams) -> dict:
    """Perform file operations with validation."""
    # Mock implementation
    if params.mode == "r":
        return {"action": "read", "path": params.path, "content": "File content here..."}
    elif params.mode == "w":
        return {"action": "write", "path": params.path, "status": "success"}
    else:
        return {"action": "append", "path": params.path, "status": "success"}

@tool(registry=validated_registry)
def query_data(params: DataQueryParams) -> list:
    """Query data with validated parameters."""
    # Mock implementation
    results = []
    for i in range(min(params.limit, 3)):
        results.append({
            "id": i + 1,
            "table": params.table,
            "data": f"Row {i + 1} from {params.table}"
        })
    
    if params.order_by:
        results.append({"note": f"Ordered by {params.order_by}"})
    
    return results

# Test with valid parameters
print("Valid file operation:")
file_params = FileOperationParams(path="/tmp/test.txt", mode="r")
result = file_operation(file_params)
print(json.dumps(result, indent=2))

print("\nValid data query:")
query_params = DataQueryParams(
    table="users",
    filters={"active": True},
    limit=5,
    order_by="created_at"
)
results = query_data(query_params)
print(json.dumps(results, indent=2))

# Test validation
print("\nTesting validation:")
try:
    # This should fail - limit too high
    invalid_params = DataQueryParams(table="users", limit=200)
except Exception as e:
    print(f"Validation error caught: {e}")

## Tool Categories and Organization

Organize tools into categories for better management:

In [ ]:
class CategorizedToolRegistry(ToolRegistry):
    """Registry that organizes tools by category."""
    
    def __init__(self):
        super().__init__()
        self.categories = {}
    
    def register_with_category(self, tool: Tool, category: str):
        """Register a tool with a category."""
        self.register(tool)
        
        if category not in self.categories:
            self.categories[category] = []
        self.categories[category].append(tool.name)
    
    def get_tools_by_category(self, category: str) -> list[Tool]:
        """Get all tools in a category."""
        tool_names = self.categories.get(category, [])
        return [self.get(name) for name in tool_names if self.get(name)]
    
    def list_categories(self) -> list[str]:
        """List all categories."""
        return list(self.categories.keys())

# Create categorized registry
cat_registry = CategorizedToolRegistry()

# Math tools
@tool(registry=cat_registry)
def add(a: float, b: float) -> float:
    """Add two numbers."""
    return a + b

@tool(registry=cat_registry)
def multiply(a: float, b: float) -> float:
    """Multiply two numbers."""
    return a * b

# String tools
@tool(registry=cat_registry)
def reverse_string(text: str) -> str:
    """Reverse a string."""
    return text[::-1]

@tool(registry=cat_registry)
def word_count(text: str) -> int:
    """Count words in text."""
    return len(text.split())

# Date tools
@tool(registry=cat_registry)
def days_between(date1: str, date2: str) -> int:
    """Calculate days between two dates (YYYY-MM-DD)."""
    d1 = datetime.strptime(date1, "%Y-%m-%d")
    d2 = datetime.strptime(date2, "%Y-%m-%d")
    return abs((d2 - d1).days)

# Manually categorize existing tools
cat_registry.categories["math"] = ["add", "multiply"]
cat_registry.categories["string"] = ["reverse_string", "word_count"]
cat_registry.categories["date"] = ["days_between"]

# Display tools by category
print("Tools by Category:")
for category in cat_registry.list_categories():
    print(f"\n{category.upper()}:")
    tools = cat_registry.get_tools_by_category(category)
    for tool in tools:
        print(f"  - {tool.name}: {tool.description}")

# Use tools from specific category
print("\nUsing math tools:")
math_tools = cat_registry.get_tools_by_category("math")
for tool in math_tools:
    if tool.name == "add":
        result = tool.execute(a=5, b=3)
        print(f"add(5, 3) = {result}")
    elif tool.name == "multiply":
        result = tool.execute(a=4, b=7)
        print(f"multiply(4, 7) = {result}")

## Advanced Tool Patterns

Create sophisticated tools with advanced patterns:

In [ ]:
# 1. Stateful Tools
class StatefulTool:
    """Tool that maintains state between calls."""
    
    def __init__(self):
        self.history = []
        self.state = {}
    
    def create_tool(self):
        """Create a stateful tool."""
        def stateful_function(action: str, key: str = None, value: str = None) -> dict:
            """Stateful operations: set, get, list, clear."""
            self.history.append({"action": action, "key": key, "value": value})
            
            if action == "set" and key and value:
                self.state[key] = value
                return {"status": "set", "key": key, "value": value}
            elif action == "get" and key:
                return {"status": "get", "key": key, "value": self.state.get(key)}
            elif action == "list":
                return {"status": "list", "state": self.state}
            elif action == "clear":
                self.state.clear()
                return {"status": "cleared"}
            else:
                return {"status": "error", "message": "Invalid action"}
        
        return Tool(
            name="stateful_store",
            description="Store and retrieve stateful data",
            func=stateful_function
        )

# Create and test stateful tool
stateful = StatefulTool()
stateful_tool = stateful.create_tool()

print("Testing stateful tool:")
print(stateful_tool.execute(action="set", key="user", value="Alice"))
print(stateful_tool.execute(action="set", key="score", value="100"))
print(stateful_tool.execute(action="list"))
print(stateful_tool.execute(action="get", key="user"))

# 2. Composite Tools
def create_composite_tool(tools: list[Tool]) -> Tool:
    """Create a tool that combines multiple tools."""
    def composite_function(tool_name: str, **kwargs) -> dict:
        """Execute one of the available tools."""
        for tool in tools:
            if tool.name == tool_name:
                return {"tool": tool_name, "result": tool.execute(**kwargs)}
        
        available = [t.name for t in tools]
        return {"error": f"Tool '{tool_name}' not found. Available: {available}"}
    
    return Tool(
        name="multi_tool",
        description="Execute multiple tools through one interface",
        func=composite_function
    )

# Create composite tool
calc_tool = Tool("calc", "Calculate", func=lambda op, a, b: eval(f"{a} {op} {b}"))
format_tool = Tool("format", "Format text", func=lambda text, style: f"[{style.upper()}] {text}")

composite = create_composite_tool([calc_tool, format_tool])

print("\nTesting composite tool:")
print(composite.execute(tool_name="calc", op="+", a=10, b=5))
print(composite.execute(tool_name="format", text="Hello", style="bold"))

# 3. Async-like Tools (simulated)
class AsyncTool:
    """Tool that simulates async operations."""
    
    def __init__(self):
        self.tasks = {}
        self.task_id = 0
    
    def create_async_tool(self):
        def async_operation(action: str, task_id: int = None, data: str = None) -> dict:
            """Simulate async operations: start, check, get_result."""
            if action == "start":
                self.task_id += 1
                self.tasks[self.task_id] = {
                    "status": "running",
                    "data": data,
                    "result": None
                }
                return {"task_id": self.task_id, "status": "started"}
            
            elif action == "check" and task_id:
                if task_id in self.tasks:
                    # Simulate completion
                    self.tasks[task_id]["status"] = "completed"
                    self.tasks[task_id]["result"] = f"Processed: {self.tasks[task_id]['data']}"
                    return {"task_id": task_id, "status": self.tasks[task_id]["status"]}
                return {"error": "Task not found"}
            
            elif action == "get_result" and task_id:
                if task_id in self.tasks:
                    return self.tasks[task_id]
                return {"error": "Task not found"}
            
            return {"error": "Invalid action"}
        
        return Tool(
            name="async_processor",
            description="Simulate async processing",
            func=async_operation
        )

# Test async tool
async_tool_manager = AsyncTool()
async_tool = async_tool_manager.create_async_tool()

print("\nTesting async tool:")
# Start task
start_result = async_tool.execute(action="start", data="Important data")
print(f"Started: {start_result}")

task_id = start_result["task_id"]

# Check status
print(f"Checking: {async_tool.execute(action='check', task_id=task_id)}")

# Get result
print(f"Result: {async_tool.execute(action='get_result', task_id=task_id)}")

## Tool Composition and Chaining

Combine tools to create more complex functionality:

In [ ]:
# Create a pipeline of tools
class ToolPipeline:
    """Chain tools together in a pipeline."""
    
    def __init__(self):
        self.steps = []
    
    def add_step(self, tool: Tool, params_mapper=None):
        """Add a tool to the pipeline.
        
        Args:
            tool: The tool to add
            params_mapper: Function to map previous result to tool params
        """
        self.steps.append((tool, params_mapper))
        return self
    
    def run(self, initial_input):
        """Run the pipeline."""
        result = initial_input
        
        for tool, mapper in self.steps:
            # Map result to parameters
            if mapper:
                params = mapper(result)
            else:
                params = result if isinstance(result, dict) else {"input": result}
            
            # Execute tool
            result = tool.execute(**params)
        
        return result

# Create tools for pipeline
extract_tool = Tool(
    name="extract_numbers",
    description="Extract numbers from text",
    func=lambda text: {"numbers": [int(s) for s in text.split() if s.isdigit()]}
)

sum_tool = Tool(
    name="sum_numbers",
    description="Sum a list of numbers",
    func=lambda numbers: {"sum": sum(numbers)}
)

format_tool = Tool(
    name="format_result",
    description="Format the result",
    func=lambda sum: f"The total is: {sum}"
)

# Create pipeline
pipeline = ToolPipeline()
pipeline.add_step(extract_tool)
pipeline.add_step(sum_tool, lambda r: {"numbers": r["numbers"]})
pipeline.add_step(format_tool, lambda r: {"sum": r["sum"]})

# Run pipeline
result = pipeline.run("I have 10 apples, 20 oranges, and 15 bananas")
print(f"Pipeline result: {result}")

# Create a more complex pipeline
print("\nComplex Pipeline Example:")

# Analysis pipeline
text_analysis_pipeline = ToolPipeline()

# Step 1: Clean text
clean_tool = Tool(
    "clean_text",
    "Clean and normalize text",
    lambda text: {
        "original": text,
        "cleaned": text.lower().strip().replace("  ", " ")
    }
)

# Step 2: Analyze
analyze_tool = Tool(
    "analyze_text",
    "Analyze text properties",
    lambda cleaned, original: {
        "cleaned": cleaned,
        "word_count": len(cleaned.split()),
        "char_count": len(cleaned),
        "unique_words": len(set(cleaned.split()))
    }
)

# Step 3: Generate report
report_tool = Tool(
    "generate_report",
    "Generate analysis report",
    lambda **stats: f"""Text Analysis Report:
- Words: {stats['word_count']}
- Characters: {stats['char_count']}
- Unique words: {stats['unique_words']}
- Lexical diversity: {stats['unique_words']/stats['word_count']:.2%}"""
)

text_analysis_pipeline.add_step(clean_tool)
text_analysis_pipeline.add_step(analyze_tool, lambda r: r)
text_analysis_pipeline.add_step(report_tool, lambda r: r)

# Run analysis
sample_text = "The quick brown fox jumps over the lazy dog. The dog was really lazy!"
analysis_result = text_analysis_pipeline.run(sample_text)
print(analysis_result)

## OpenAI Function Format

Convert tools to OpenAI function calling format:

In [ ]:
# Create a registry with various tools
openai_registry = ToolRegistry()

# Define tools with proper type hints and descriptions
@tool(registry=openai_registry)
def get_stock_price(symbol: str, exchange: str = "NYSE") -> dict:
    """Get current stock price for a symbol.
    
    Args:
        symbol: Stock ticker symbol (e.g., AAPL, GOOGL)
        exchange: Stock exchange (NYSE, NASDAQ, etc.)
        
    Returns:
        Dictionary with price information
    """
    # Mock implementation
    prices = {"AAPL": 185.50, "GOOGL": 142.30, "MSFT": 378.90}
    price = prices.get(symbol, 100.0)
    return {
        "symbol": symbol,
        "price": price,
        "exchange": exchange,
        "currency": "USD",
        "timestamp": datetime.now().isoformat()
    }

@tool(registry=openai_registry)
def calculate_investment(amount: float, symbol: str, years: int) -> dict:
    """Calculate investment returns.
    
    Args:
        amount: Investment amount in USD
        symbol: Stock symbol to invest in
        years: Number of years to hold
        
    Returns:
        Investment projection
    """
    # Mock calculation
    annual_return = 0.08  # 8% annual return
    final_amount = amount * (1 + annual_return) ** years
    
    return {
        "initial_amount": amount,
        "final_amount": round(final_amount, 2),
        "total_return": round(final_amount - amount, 2),
        "return_percentage": round((final_amount - amount) / amount * 100, 2),
        "years": years,
        "symbol": symbol
    }

# Convert to OpenAI format
print("OpenAI Function Definitions:")
print("=" * 50)

functions = openai_registry.to_openai_functions()
for func in functions:
    print(f"\nFunction: {func['name']}")
    print(f"Description: {func['description']}")
    print("Parameters:")
    print(json.dumps(func['parameters'], indent=2))

# Simulate OpenAI function calling
print("\n" + "=" * 50)
print("Simulating function calls:")

# Example function call
function_call = {
    "name": "get_stock_price",
    "arguments": json.dumps({"symbol": "AAPL"})
}

# Execute the function
args = json.loads(function_call["arguments"])
result = openai_registry.execute_tool(function_call["name"], **args)
print(f"\nFunction call: {function_call}")
print(f"Result: {json.dumps(result, indent=2)}")

## Real-World Example: Multi-Tool System

Build a comprehensive tool system for a virtual assistant:

In [ ]:
class VirtualAssistantTools:
    """Complete tool system for a virtual assistant."""
    
    def __init__(self):
        self.registry = ToolRegistry()
        self.user_preferences = {}
        self.task_history = []
        self._register_all_tools()
    
    def _register_all_tools(self):
        """Register all assistant tools."""
        
        # Calendar tools
        @tool(registry=self.registry)
        def schedule_meeting(title: str, date: str, time: str, duration: int = 60) -> dict:
            """Schedule a meeting."""
            meeting = {
                "id": len(self.task_history) + 1,
                "title": title,
                "date": date,
                "time": time,
                "duration": duration,
                "status": "scheduled"
            }
            self.task_history.append({"type": "meeting", "data": meeting})
            return meeting
        
        @tool(registry=self.registry)
        def check_calendar(date: str) -> list:
            """Check calendar for a specific date."""
            events = []
            for task in self.task_history:
                if task["type"] == "meeting" and task["data"]["date"] == date:
                    events.append(task["data"])
            return events
        
        # Task management
        @tool(registry=self.registry)
        def create_task(title: str, priority: str = "medium", due_date: str = None) -> dict:
            """Create a new task."""
            task = {
                "id": len(self.task_history) + 1,
                "title": title,
                "priority": priority,
                "due_date": due_date,
                "status": "pending",
                "created_at": datetime.now().isoformat()
            }
            self.task_history.append({"type": "task", "data": task})
            return task
        
        @tool(registry=self.registry)
        def list_tasks(status: str = "all") -> list:
            """List tasks by status."""
            tasks = []
            for item in self.task_history:
                if item["type"] == "task":
                    if status == "all" or item["data"]["status"] == status:
                        tasks.append(item["data"])
            return tasks
        
        # Preferences
        @tool(registry=self.registry)
        def set_preference(key: str, value: str) -> dict:
            """Set user preference."""
            self.user_preferences[key] = value
            return {"key": key, "value": value, "status": "saved"}
        
        @tool(registry=self.registry)
        def get_preference(key: str) -> str:
            """Get user preference."""
            return self.user_preferences.get(key, "Not set")
        
        # Information tools
        @tool(registry=self.registry)
        def get_weather(location: str) -> dict:
            """Get weather for location."""
            # Mock weather data
            weather_data = {
                "location": location,
                "temperature": "22°C",
                "condition": "Partly cloudy",
                "humidity": "65%",
                "forecast": "Mild with occasional clouds"
            }
            return weather_data
        
        @tool(registry=self.registry)
        def search_knowledge(query: str, category: str = "general") -> dict:
            """Search knowledge base."""
            # Mock knowledge search
            return {
                "query": query,
                "category": category,
                "results": [
                    f"Information about {query} in {category}",
                    f"Related topics to {query}",
                    f"Common questions about {query}"
                ],
                "confidence": 0.85
            }
    
    def get_tool_summary(self) -> dict:
        """Get summary of available tools."""
        categories = {
            "calendar": ["schedule_meeting", "check_calendar"],
            "tasks": ["create_task", "list_tasks"],
            "preferences": ["set_preference", "get_preference"],
            "information": ["get_weather", "search_knowledge"]
        }
        
        summary = {}
        for category, tool_names in categories.items():
            summary[category] = []
            for name in tool_names:
                tool = self.registry.get(name)
                if tool:
                    summary[category].append({
                        "name": tool.name,
                        "description": tool.description
                    })
        
        return summary

# Create and test the assistant
assistant = VirtualAssistantTools()

print("Virtual Assistant Tools:")
print("=" * 50)
tool_summary = assistant.get_tool_summary()
for category, tools in tool_summary.items():
    print(f"\n{category.upper()}:")
    for tool in tools:
        print(f"  - {tool['name']}: {tool['description']}")

# Demonstrate usage
print("\n" + "=" * 50)
print("Demonstrating Assistant Capabilities:\n")

# Set preferences
assistant.registry.execute_tool("set_preference", key="work_hours", value="9am-5pm")
assistant.registry.execute_tool("set_preference", key="timezone", value="PST")

# Schedule meetings
meeting1 = assistant.registry.execute_tool(
    "schedule_meeting",
    title="Team Standup",
    date="2024-01-15",
    time="09:00",
    duration=30
)
print(f"Scheduled: {meeting1}")

# Create tasks
task1 = assistant.registry.execute_tool(
    "create_task",
    title="Review project proposal",
    priority="high",
    due_date="2024-01-16"
)
print(f"\nCreated task: {task1}")

# Check calendar
events = assistant.registry.execute_tool("check_calendar", date="2024-01-15")
print(f"\nCalendar for 2024-01-15: {events}")

# Get weather
weather = assistant.registry.execute_tool("get_weather", location="San Francisco")
print(f"\nWeather: {weather}")

# List all tasks
all_tasks = assistant.registry.execute_tool("list_tasks")
print(f"\nAll tasks: {all_tasks}")

## Best Practices

Guidelines for creating effective tools:

In [ ]:
# 1. Clear and descriptive names
@tool
def calculate_compound_interest(principal: float, rate: float, years: int) -> float:
    """Calculate compound interest.
    
    Args:
        principal: Initial amount
        rate: Annual interest rate (as decimal, e.g., 0.05 for 5%)
        years: Number of years
        
    Returns:
        Final amount after compound interest
    """
    return principal * (1 + rate) ** years

# 2. Proper error handling
@tool
def safe_divide(a: float, b: float) -> float:
    """Safely divide two numbers."""
    if b == 0:
        raise ValueError("Cannot divide by zero")
    return a / b

# 3. Return structured data
@tool
def analyze_text_sentiment(text: str) -> dict:
    """Analyze sentiment of text."""
    # Mock analysis
    positive_words = ["good", "great", "excellent", "love", "happy"]
    negative_words = ["bad", "terrible", "hate", "sad", "awful"]
    
    text_lower = text.lower()
    positive_count = sum(1 for word in positive_words if word in text_lower)
    negative_count = sum(1 for word in negative_words if word in text_lower)
    
    if positive_count > negative_count:
        sentiment = "positive"
        score = positive_count / (positive_count + negative_count) if negative_count > 0 else 1.0
    elif negative_count > positive_count:
        sentiment = "negative"
        score = negative_count / (positive_count + negative_count) if positive_count > 0 else 1.0
    else:
        sentiment = "neutral"
        score = 0.5
    
    return {
        "text": text,
        "sentiment": sentiment,
        "confidence": score,
        "positive_words_found": positive_count,
        "negative_words_found": negative_count
    }

# 4. Use type hints and validation
from typing import List, Union

@tool
def process_data_batch(
    data: List[Union[int, float]], 
    operation: str = "sum"
) -> Union[float, List[float]]:
    """Process a batch of numerical data.
    
    Args:
        data: List of numbers to process
        operation: Operation to perform (sum, mean, max, min, normalize)
        
    Returns:
        Result of the operation
    """
    if not data:
        raise ValueError("Data list cannot be empty")
    
    if operation == "sum":
        return sum(data)
    elif operation == "mean":
        return sum(data) / len(data)
    elif operation == "max":
        return max(data)
    elif operation == "min":
        return min(data)
    elif operation == "normalize":
        min_val = min(data)
        max_val = max(data)
        if max_val == min_val:
            return [0.5] * len(data)
        return [(x - min_val) / (max_val - min_val) for x in data]
    else:
        raise ValueError(f"Unknown operation: {operation}")

# Test best practice examples
print("Testing best practice tools:")
print(f"Compound interest: ${calculate_compound_interest(1000, 0.05, 10):.2f}")
print(f"Sentiment: {analyze_text_sentiment('This is a great product! I love it.')}")
print(f"Data processing: {process_data_batch([1, 2, 3, 4, 5], 'mean')}")

## Summary

In this tutorial, you learned:

- ✅ How to create tools with the @tool decorator
- ✅ How to manually register tools for more control
- ✅ How to use Pydantic for parameter validation
- ✅ How to organize tools into categories
- ✅ How to create stateful and composite tools
- ✅ How to chain tools together in pipelines
- ✅ How to convert tools to OpenAI function format
- ✅ Best practices for tool design

Tools are essential for:
- Extending agent capabilities beyond text generation
- Integrating with external systems and APIs
- Performing calculations and data processing
- Building interactive AI applications

## Next Steps

Continue with:

- **Tutorial 8: Agents** - Build autonomous agents that use tools
- **Tutorial 9: Advanced Features** - Async operations and more
- **Tutorial 10: Real-World Examples** - Complete applications

Happy tool building! 🛠️